In [1]:
##########################################################################
## Email Manager
##########################################################################

"""
Custom email manager to for stripe invoices
"""

##########################################################################
## Imports
##########################################################################

from mail_templated import EmailMessage
from backend.utility import *
from subscription.models import SubscriptionMember, SubscriptionPlan
from backend.fees import AnorakFeeManager
from djstripe.models import Customer

anorakFeeManager = AnorakFeeManager()

##########################################################################
## Email Manager
##########################################################################

class EmailManager(object):
    
    def __init__(self, user, invoice=None):
        self.invoice = invoice
        if invoice == None:
            self.invoice = user.upcoming_invoice()
        self.invoiceData = self._clean_invoice_data(invoice)
        self.customer = self._get_customer_from_invoice()
        self.user = self.customer.subscriber
        
    def _clean_invoice_data(self, invoice):
        invoiceData = invoice.lines.data
        return [item for item in invoiceData if item.amount!=0]
    
    def _invoice_data(self):
        return self.invoice.lines.data
    
    def _invoice_items_from_product_name(self, productName):
        invoiceData = self.invoiceData
        items = self.find_items([productName])
        return items
    
    def _get_item_name(self, item):
        if item.plan:
            member = self._member_from_item(item)
            return member.subscription_account.subscription_plan.product_name
        return item.description
    
    def _format_val(self, value):
        if value != None:
            return round((value/100),2)
        return 0
    
    def _is_item_refund(self, item):
        prorated_amount = self._get_item_prorated_amount(item)
        wasRefund = False
        if (prorated_amount and prorated_amount < 0):
            wasRefund = True
        return wasRefund
    
    def _get_item_prorated_amount(self, item):
        if self._is_prorated(item) and item.plan:
            return self._format_val(item.amount)
        return None
    
    def _is_prorated(self, item):
        itemPrice = item.amount
        planPrice = self._get_item_plan_amount(item)
        return itemPrice != planPrice
    
    def _get_item_plan_amount(self, item):
        if item.plan:
            if item.amount < 0:
                return -1 * self._format_val(item.plan.amount)
            return self._format_val(item.plan.amount)
        return self._format_val(item.amount)
    
    def _get_invoice_billing_date_time(self):
        return convert_epoch(self.invoice.period_end)
    
    def _get_invoice_billing_date(self):
        billingDateTime = self._get_invoice_billing_date_time()
        return date_time_to_date(billingDateTime)
    
    def _get_invoice_end_date_time(self):
        date = self._get_invoice_billing_date_time()
        lastDayOfMonth = days_in_a_month(date)
        return datetime.strptime('{0} {1} {2}'.format(
            date.month, 
            lastDayOfMonth, 
            date.year), '%m %d %Y'
        )
    
    def _current_date(self):
        currentDateTime = convert_epoch(get_current_epoch())
        return date_time_to_date(currentDateTime)
    
    def _get_invoice_end_date(self):
        endDateTime = self._get_invoice_end_date_time()
        return date_time_to_date(endDateTime)
    
    def _get_invoice_renewal_date_time(self):
        endDateTime = self._get_invoice_end_date_time()
        return get_first_day_of_next_month(endDateTime)
    
    def _get_invoice_renewal_date(self):
        renewalDateTime = self._get_invoice_renewal_date_time()
        return date_time_to_date(renewalDateTime)
    
    def _get_invoice_start_date_time(self, item):
        print(item)
        if item:
            return item.period.start
        least = self.invoiceData[0].period.start
        for item in self.invoiceData:
            if item.period.start < least:
                least = item.period.start
        return least
    
    def _get_invoice_start_date(self, item):
        startDateEpoch = self._get_invoice_start_date_time(item)
        startDateTime = convert_epoch(startDateEpoch)
        return date_time_to_date(startDateTime)
    
    def _item_plan_description(self, item):
        if not item.plan:
            return item.description
        monthOf = self._get_invoice_billing_date_time()
        #member = self._member_from_item(item)
        try:
            subscriptionPlan = SubscriptionPlan.objects.get(stripe_plan_id=item.plan.id)
            plan = subscriptionPlan.product_name
            return  plan+" - Month of "+monthOf.strftime('%B')
        except:
            print("CANT FIND")
            print(item)
            return "Subscription"
    
    def _item_prorated_description(self, item):
        if not item.plan:
            return None
        return item.description
    
    def _was_item_canceled(self, item):
        if self._is_prorated(item) and item.plan and item.amount <= 0:
            return True
        return False
    
    def _item_dictionary(self, item):
        proratedDescription = None
        proratedAmount = None
        if item.proration:
            proratedDescription = self._item_prorated_description(item)
            proratedAmount = self._get_item_prorated_amount(item)
        return({
            'item_id': item.id,
            'plan_description': self._item_plan_description(item),
            'prorated_description': proratedDescription,
            'plan_amount': self._get_item_plan_amount(item),
            'prorated_amount': proratedAmount,
            'was_refunded': self._is_item_refund(item)
        })
    
    def _member_from_item(self, item):
        plan = item.plan.id
        return SubscriptionMember.objects.get(
            subscription_account__subscription_plan__stripe_plan_id=plan,
            user = self.user
        )
    
    def _get_customer_from_invoice(self):
        return Customer.objects.get(id=self.invoice.customer)
    
    def _get_shipping_data(self):
        customerAPI = self.customer.api_retrieve()
        return customerAPI.shipping
    
    def _get_card_last4(self):
        customerAPI = self.customer.api_retrieve()
        return customerAPI.default_source.last4
    
    def _get_user_address_dict(self):
        shippingData = self._get_shipping_data()
        addressData = shippingData.address
        return({
            'line1': addressData.line1,
            'line2': addressData.line2,
            'zip': addressData.postal_code,
            'state': addressData.state,
            'city': addressData.city,
        })
        return shippingData
    
    def _init_invoice_dict(self, item=None):
        dictionary = {}
        dictionary['items']={}
        dictionary['invoice_number'] = self.invoice.number
        dictionary['billing_date'] = self._get_invoice_billing_date()
        dictionary['renewal_date'] = self._get_invoice_renewal_date()
        dictionary['end_date'] = self._get_invoice_end_date()
        dictionary['start_date'] = self._get_invoice_start_date(item)
        dictionary['address'] = self._get_user_address_dict()
        dictionary['last4'] = self._get_card_last4()
        return dictionary
       
    def _get_item_total(self, invoiceDictionaryItem):
        itemTotal = invoiceDictionaryItem['plan_amount']
        if invoiceDictionaryItem['prorated_amount']:
            itemTotal += invoiceDictionaryItem['prorated_amount']
        return itemTotal
    
    def _get_invoice_subtotal_from_dict(self, invoiceDictionary):
        total = 0.0
        for invoiceDictionaryItem in invoiceDictionary['items']:
            itemTotal = self._get_item_total(invoiceDictionaryItem)
            total += itemTotal
        return total
    
    def _get_invoice_items(self, receiptItem):
        items = []
        anorakFee = None
        if receiptItem:
            item = self._item_dictionary(receiptItem)
            if anorakFeeManager.feeDescription == receiptItem.description :
                anorakFee = item
            elif item:
                items.append(item)
        else:
            for item in self.invoiceData:
                if anorakFeeManager.feeDescription != item.description and item:
                    items.append(self._item_dictionary(item))
                elif item:
                    anorakFee = self._item_dictionary(item)
        if anorakFee:
            items.append(anorakFee)
            return items, True 
        return items, False
    
    def _get_invoice_type(self, receiptItem):
        invoiceType = 'multiple'
        if receiptItem:
            invoiceType = 'single'
        return invoiceType
    
    def _get_tax_amount(self, subtotal, taxPercent):
        return self._format_val(taxPercent * subtotal)
    
    def invoice_to_dict(self, receiptItem=None):
        dictionary = self._init_invoice_dict(receiptItem)
        taxPercent = self.invoice.tax_percent
        items, hasAnorakFee = self._get_invoice_items(receiptItem)
        dictionary['items'] = items
        subtotal = self._get_invoice_subtotal_from_dict(dictionary)
        taxAmount = self._get_tax_amount(subtotal, taxPercent)
        dictionary['subtotal'] = '{:.2f}'.format(subtotal)
        dictionary['tax'] = '{:.2f}'.format(taxAmount)
        dictionary['total'] = '{:.2f}'.format(round((subtotal + taxAmount),2))
        dictionary['tax_percent'] = '{:.2f}'.format(taxPercent)
        dictionary['type'] = self._get_invoice_type(receiptItem)
        dictionary['has_anorak_fee'] = hasAnorakFee
        return dictionary
        
    def find_items(self, search):
        search = [term.lower() for term in search]
        found = []
        for item in self.invoiceData:
            description = item.description.lower()
            if all(term in description for term in search):
                found.append(item)
        return found
    
    def email_receipt(self, receiptItem=None):
        dictionary = self.invoice_to_dict(receiptItem=receiptItem)  
        message = EmailMessage('receipt.tpl', 
            {'user': self.user, 'data':dictionary}, 
            'Anorak@ianorak.com', 
            to =[self.user.email]
        )
        #message.send()
        return dictionary
    
    def email_refund(self, refundItem):
        dictionary = self.invoice_to_dict(receiptItem=refundItem)   
        cancelDate = self._current_date()
        message = EmailMessage('refund.tpl', 
            {'user': self.user, 'data':dictionary, 'date_cancelled':cancelDate}, 
            'Anorak@ianorak.com', 
            to =[self.user.email]
        )
        message.send()
        return dictionary

In [2]:
"""
Custom invoice manager to for stripe invoices
"""

##########################################################################
## Imports
##########################################################################

import sys
from backend.utility import *
from subscription.models import SubscriptionMember

##########################################################################
## Invoice Manager
##########################################################################

class InvoiceManager(object):
    
    def _get_invoice_items(self, invoice):
        return invoice.lines.data
    
    def _get_plan_id_from_member(self, member):
        return self._get_invoice_item_from_member(member).stripe_plan_id
    
    def _datetime_to_epoch(self, dateTime):
        return int(dateTime.timestamp())
    
    def _time_difference(self, epoch1, epoch2):
        return abs(epoch2 - epoch1)
    
    def _get_invoice_item_from_member(self, member):
        return member.subscription_account.subscription_plan
    
    def _item_plan_matches(self, item, memberPlan):
        if item.plan and item.plan.id == memberPlan:
            return True
        return False    
        
    def _get_closest_item(self, member, invoice, memberPlan):
        items = self._get_invoice_items(invoice)
        leastDifference = self._initial_difference(items, member)
        closestItem = items[0]
        for item in items:
            difference = self._check_item_distance(item, member, memberPlan)
            if difference < leastDifference:
                leastDifference = difference
                closestItem = item
        return closestItem
    
    def _check_item_distance(self, item, member, memberPlan):
        if self._item_plan_matches(item, memberPlan):
            return self._get_item_difference(item, member)
        return sys.maxsize
            
    def _get_member_create_epoch(self, member):
        createdDateTime = member.date_created
        createdEpoch = self._datetime_to_epoch(createdDateTime)
        return createdEpoch
            
    def _get_item_difference(self, item, member):
        memberCreated = self._get_member_create_epoch(member)
        itemCreated = item.period.start
        return self._time_difference(itemCreated, memberCreated)
    
    def _initial_difference(self, items, member):
        initialItem = items[0]
        return self._get_item_difference(initialItem, member)
    
    def _event_data(self, event):
        return event.data['object']['items']['data']
    
    def _event_previous_data(self, event):
        return event.data['previous_attributes']['items']['data']
    
    def _event_new_data(self, event):
        newData = self._event_data(event)
        previousData = self._event_previous_data(event)
        return [x for x in newData + previousData if x not in newData or x not in previousData][0]
    
    def get_invoice_subscription_item(self, newSubscriptionItemId, invoice):
        invoiceItem = None
        for item in self._get_invoice_items(invoice):
            if('subscription_item' in item):
                subscriptionItemId = item['subscription_item']
                if(subscriptionItemId == newSubscriptionItemId):
                    invoiceItem = item
        return invoiceItem
    
    def get_new_subscription_item_id(self, event):
        return self._event_new_data(event)['id']
    
        
    def get_invoice_item(self, event, invoice):
        newSubscriptionItemId = self.get_new_subscription_item_id(event)
        newSubscriptionLineItem = self.get_invoice_subscription_item(newSubscriptionItemId, invoice)
        planId, productId = newSubscriptionLineItem.plan.id, newSubscriptionLineItem.plan.product
        invoiceData = invoice.lines.data
        for item in invoiceData:
            if item.plan:
                cond1 = item.plan.id == planId
                cond2 = item.plan.product == productId 
                cond3 = item.subscription_item == newSubscriptionItemId
                if cond1 and cond2 and cond3:
                    return item
        return None
    
    def get_closest_item(self, member, invoice=None):
        if invoice == None:
            invoice = member.user.upcoming_invoice()
        memberPlan = self._get_plan_id_from_member(member)
        return self._get_closest_item(member, invoice, memberPlan)

In [3]:
invoiceManager = InvoiceManager()

In [4]:
user = User.objects.get(email='veyorokon@gmail.com')
# invoice = user.upcoming_invoice()
em = EmailManager(user, invoice)
notification = EmailReceiptNotification.objects.get(pk=3)
instance = notification.trigger_event

In [6]:
invoiceItem = invoiceManager.get_invoice_item(instance, invoice)

In [7]:
em.email_receipt(invoiceItem)

{
  "amount": 126,
  "currency": "usd",
  "description": "Remaining time on Netflix - Individual Plan after 27 Mar 2019",
  "discountable": false,
  "id": "ii_1EIRgqGjkoMLHlzNAcwYH5nS",
  "invoice_item": "ii_1EIRgqGjkoMLHlzNAcwYH5nS",
  "livemode": false,
  "metadata": {},
  "object": "line_item",
  "period": {
    "end": 1554076800,
    "start": 1553654432
  },
  "plan": {
    "active": true,
    "aggregate_usage": null,
    "amount": 799,
    "billing_scheme": "per_unit",
    "created": 1553648843,
    "currency": "usd",
    "id": "plan_ElyBy6j0nGmQek",
    "interval": "month",
    "interval_count": 1,
    "livemode": false,
    "metadata": {},
    "nickname": "$7.99 billed monthly",
    "object": "plan",
    "product": "prod_ElyBsFa2v8qS2m",
    "tiers": null,
    "tiers_mode": null,
    "transform_usage": null,
    "trial_period_days": null,
    "usage_type": "licensed"
  },
  "proration": true,
  "quantity": 1,
  "subscription": "sub_Ely4vTmySHl9Tw",
  "subscription_item": "si_Elz

{'items': [{'item_id': 'ii_1EIRgqGjkoMLHlzNAcwYH5nS',
   'plan_description': 'Netflix - Individual Plan - Month of April',
   'prorated_description': 'Remaining time on Netflix - Individual Plan after 27 Mar 2019',
   'plan_amount': 7.99,
   'prorated_amount': 1.26,
   'was_refunded': False}],
 'invoice_number': '0D37A1A-0002',
 'billing_date': 'April 01, 2019',
 'renewal_date': 'May 01, 2019',
 'end_date': 'April 30, 2019',
 'start_date': 'March 27, 2019',
 'address': {'line1': '1600 Villa Street',
  'line2': 'Apt. 119',
  'zip': '94041',
  'state': 'CA',
  'city': 'Mountain View'},
 'last4': '4242',
 'subtotal': '9.25',
 'tax': '0.58',
 'total': '9.83',
 'tax_percent': '6.25',
 'type': 'single',
 'has_anorak_fee': False}